# ACP

In [1]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.PCA
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.43.118:4040
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1546162431503)
SparkSession available as 'spark'


2018-12-30 10:33:41 WARN  Utils:66 - Your hostname, medard resolves to a loopback address: 127.0.1.1; using 192.168.43.118 instead (on interface wlp3s0b1)
2018-12-30 10:33:41 WARN  Utils:66 - Set SPARK_LOCAL_IP if you need to bind to another address
2018-12-30 10:33:44 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.feature.PCA
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.feature.StandardScaler


In [16]:
val file ="Machine_Learning_Sections/PCA/Cancer_Data"
val df = spark.read.option("header","true").option("inferSchema","true").format("csv").load(file)
val colnames = df.columns

file: String = Machine_Learning_Sections/PCA/Cancer_Data
df: org.apache.spark.sql.DataFrame = [mean radius: int, mean texture: double ... 28 more fields]
colnames: Array[String] = Array(mean radius, mean texture, mean perimeter, mean area, mean smoothness, mean compactness, mean concavity, mean concave points, mean symmetry, mean fractal dimension, radius error, texture error, perimeter error, area error, smoothness error, compactness error, concavity error, concave points error, symmetry error, fractal dimension error, worst radius, worst texture, worst perimeter, worst area, worst smoothness, worst compactness, worst concavity, worst concave points, worst symmetry, worst fractal dimension)


In [25]:
df.printSchema

root
 |-- mean radius: integer (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst

In [19]:
val assembler = new VectorAssembler().setInputCols(colnames).setOutputCol("features")
val data = assembler.transform(df).select($"features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_4689d9c1f368
data: org.apache.spark.sql.DataFrame = [features: vector]


In [20]:
data.printSchema

root
 |-- features: vector (nullable = true)



In [22]:
data.show(2)

2018-12-30 11:27:18 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
+--------------------+
|            features|
+--------------------+
|[0.0,17.99,10.38,...|
|[1.0,20.57,17.77,...|
+--------------------+
only showing top 2 rows



In [32]:
val scaler = new StandardScaler()
  .setInputCol("features")
  .setOutputCol("scaledFeatures")
  .setWithStd(true)
  .setWithMean(false)
val scalerModel = scaler.fit(data)
var scaledData = scalerModel.transform(data).select($"scaledFeatures")
scaledData = scaledData.select(scaledData("scaledFeatures").as("features"))

scaler: org.apache.spark.ml.feature.StandardScaler = stdScal_472d8ba0a555
scalerModel: org.apache.spark.ml.feature.StandardScalerModel = stdScal_472d8ba0a555
scaledData: org.apache.spark.sql.DataFrame = [features: vector]
scaledData: org.apache.spark.sql.DataFrame = [features: vector]


In [36]:
val pca = (new PCA()
  .setInputCol("features")
  .setOutputCol("pcaFeatures")
  .setK(3)
  .fit(scaledData))

pca: org.apache.spark.ml.feature.PCAModel = pca_ce3ed9f4cbcf


In [38]:
val pcaDF = pca.transform(scaledData)
val result = pcaDF.select("pcaFeatures")

pcaDF: org.apache.spark.sql.DataFrame = [features: vector, pcaFeatures: vector]
result: org.apache.spark.sql.DataFrame = [pcaFeatures: vector]


In [41]:
result.head(2)

res27: Array[org.apache.spark.sql.Row] = Array([[21.62199738236476,8.516595739466684,-3.7318474175794782]], [[15.121737034758134,2.697138979042207,-2.3546461829874357]])
